In [1]:
import pandas as pd
import numpy as np
import regex as re

In [2]:
# leer con encoding latin1
shark_df =  pd.read_csv('GSAF5.csv', encoding='latin1')

In [3]:
# primer vistazo del dataframe
shark_df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5987,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6,NaN,NaN
5988,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5,NaN,NaN
5989,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4,NaN,NaN
5990,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,...,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3,NaN,NaN


In [4]:
# revisamos las columnas
shark_df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
# las columnas Sex y Species tienen especios en blanco en el nombre
# vamos a renombrar los nombres 
shark_df.rename(columns={'Sex ':'Sex', 'Species ':'Species'}, inplace=True)

In [6]:
# revisamos las estadisticas generales con describe
shark_df.describe()

,Year,original order
count,5992.000000,5992.000000
mean,1925.204606,2997.548899
std,286.473712,1729.860210
min,0.000000,2.000000
25%,1942.000000,1499.750000
50%,1975.000000,2997.500000
75%,2003.000000,4495.250000
max,2016.000000,5993.000000


In [7]:
# empezamos hace una limpieza "inespecifica"
# por ejemplo podemos revisar si hay posibles registros duplicados
shark_df.drop_duplicates()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5987,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6,NaN,NaN
5988,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5,NaN,NaN
5989,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4,NaN,NaN
5990,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,...,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3,NaN,NaN


In [8]:
# continuamos la limpieza genérica revisando si hubiera registros con valores nulos
shark_df.dropna(axis=0,how='all',inplace=True) 

In [9]:
shark_df.dropna(axis=1,how='all',inplace=True) 

In [10]:
#pasamos a revisar columnas de forma más especifica
shark_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5992 entries, 0 to 5991
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case Number             5992 non-null   object
 1   Date                    5992 non-null   object
 2   Year                    5992 non-null   int64 
 3   Type                    5992 non-null   object
 4   Country                 5949 non-null   object
 5   Area                    5590 non-null   object
 6   Location                5496 non-null   object
 7   Activity                5465 non-null   object
 8   Name                    5792 non-null   object
 9   Sex                     5425 non-null   object
 10  Age                     3311 non-null   object
 11  Injury                  5965 non-null   object
 12  Fatal (Y/N)             5973 non-null   object
 13  Time                    2779 non-null   object
 14  Species                 3058 non-null   object
 15  Inve

In [11]:
# vemos que hay 2 columnas llamadas Unnamed: 22 y Unnamed: 23 
# cuyos valores excepto 3 son nulos
shark_df.loc[:,['Unnamed: 22','Unnamed: 23']]

,Unnamed: 22,Unnamed: 23
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
5987,NaN,NaN
5988,NaN,NaN
5989,NaN,NaN
5990,NaN,NaN


In [12]:
# como no aportan información, las eliminaremos por completo
shark_df.drop(['Unnamed: 22','Unnamed: 23'],axis=1, inplace=True)

In [13]:
# Continuamos con la columna Fatal (Y/N), que teoricamente deberia constar de 2 tipos de valores
shark_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5992 entries, 0 to 5991
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case Number             5992 non-null   object
 1   Date                    5992 non-null   object
 2   Year                    5992 non-null   int64 
 3   Type                    5992 non-null   object
 4   Country                 5949 non-null   object
 5   Area                    5590 non-null   object
 6   Location                5496 non-null   object
 7   Activity                5465 non-null   object
 8   Name                    5792 non-null   object
 9   Sex                     5425 non-null   object
 10  Age                     3311 non-null   object
 11  Injury                  5965 non-null   object
 12  Fatal (Y/N)             5973 non-null   object
 13  Time                    2779 non-null   object
 14  Species                 3058 non-null   object
 15  Inve

In [14]:
# comprobamos si solo hay valores Y o N en la columa
shark_df['Fatal (Y/N)'].unique()

array(['N', 'Y', nan, 'UNKNOWN', ' N', 'F', 'N ', '#VALUE!', 'n'],
      dtype=object)

In [15]:
# comprobamos que hay demasiada variedad en los valores
# dejaremos unicamente 3: Y, N o UNKNOWN

shark_df['Fatal (Y/N)'] = shark_df['Fatal (Y/N)'].apply(lambda x: x if type(x) == str else 'UNKNOWN')
shark_df['Fatal (Y/N)'] = shark_df['Fatal (Y/N)'].apply(lambda x: 'UNKNOWN' if x == '#VALUE!' else x )
shark_df['Fatal (Y/N)'] = shark_df['Fatal (Y/N)'].apply(lambda x: 'N' if x in [' N','N ', 'n','F'] else x)

In [16]:
# volvemos a comprobar si solo hay valores Y, N o UNKNOWN en la columa
shark_df['Fatal (Y/N)'].unique()

array(['N', 'Y', 'UNKNOWN'], dtype=object)

In [17]:
shark_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5992 entries, 0 to 5991
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case Number             5992 non-null   object
 1   Date                    5992 non-null   object
 2   Year                    5992 non-null   int64 
 3   Type                    5992 non-null   object
 4   Country                 5949 non-null   object
 5   Area                    5590 non-null   object
 6   Location                5496 non-null   object
 7   Activity                5465 non-null   object
 8   Name                    5792 non-null   object
 9   Sex                     5425 non-null   object
 10  Age                     3311 non-null   object
 11  Injury                  5965 non-null   object
 12  Fatal (Y/N)             5992 non-null   object
 13  Time                    2779 non-null   object
 14  Species                 3058 non-null   object
 15  Inve

In [18]:
# comprobamos luego las columnas href y href formula
(shark_df['href formula'] == shark_df['href']).value_counts()

True     5938
False      54
dtype: int64

In [19]:
# tras una primera inspección muchos valores parecen duplicados
# pero 54 de ellos no lo parecen. Los revisamos
shark_df[shark_df['href formula'] != shark_df['href']].loc[:,['href','href formula']]

,href,href formula
20,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
27,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
61,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
107,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
114,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
134,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
180,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
193,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
232,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
262,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


In [20]:
## vemos valores NaN en una varias registros
# copiaremos el valor de la columna no nula
shark_df.loc[5686,'href'] = shark_df['href formula'].iloc[5686]
shark_df.loc[2274,'href'] = shark_df['href formula'].iloc[2274]
shark_df.loc[193,'href'] = shark_df['href formula'].iloc[193]
shark_df.loc[3019,'href formula'] = shark_df['href'].iloc[3019]

# tambien hay dos valores que no son de tipo href
shark_df.loc[448,'href formula'] = shark_df['href'].iloc[448]
shark_df.loc[365,'href'] = shark_df['href formula'].iloc[365]

In [21]:
# comprobamos si hay menos valores repetidos
(shark_df['href formula'] == shark_df['href']).value_counts()

True     5944
False      48
dtype: int64

In [22]:
# buscamos que más diferencias hay 
shark_df['href'].iloc[20]

'http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2016.07.29-Spain.pdf'

In [23]:
shark_df['href formula'].iloc[20]

'http://sharkattackfile.net/spreadsheets/pdf_directory/2016.07.29-Spain.pdf'

In [24]:
# algunos registros parace que tienen la direccion http duplicada
# vamos a usar regex para quedarnos solo con una dirección http
one_http = re.compile('^http.*(http.*)')

for i in shark_df.loc[:,'href'].index:
    # si detectamos 2 http con regex modificamos el valor
    if len(one_http.findall(shark_df.loc[i,'href'])) > 0:
        unique_http = one_http.findall(shark_df.loc[i,'href'])
        shark_df.loc[i,'href'] = unique_http[0]

for i in shark_df.loc[:,'href formula'].index:
    if len(one_http.findall(shark_df.loc[i,'href formula'])) > 0:
        unique_http = one_http.findall(shark_df.loc[i,'href formula'])
        shark_df.loc[i,'href formula'] = unique_http[0]
                                                                                       

In [25]:
# comprobamos si hay menos valores repetidos
(shark_df['href formula'] == shark_df['href']).value_counts()

True     5959
False      33
dtype: int64

In [26]:
# revisamos otra vez para ver que más problemas puede haber en la columna
shark_df[shark_df['href formula'] != shark_df['href']].loc[:,['href','href formula']]

,href,href formula
27,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
134,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
180,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
263,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
264,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
271,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
272,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
293,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
323,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
361,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


In [27]:
# quitaremos los espaciones en blanco de los nombres
shark_df['href'].iloc[4394]

'http://sharkattackfile.net/spreadsheets/pdf_directory/1945.07.30 - USS Indianapolis.pdf'

In [28]:
shark_df['href formula'].iloc[4394]

'http://sharkattackfile.net/spreadsheets/pdf_directory/1945.07.30-USS Indianapolis.pdf'

In [29]:
for i in shark_df[shark_df['href formula'] != shark_df['href']]['href'].index:
    shark_df.loc[i,'href'] = shark_df.loc[i,'href'].replace(' ','',)

In [30]:
for i in shark_df[shark_df['href formula'] != shark_df['href']]['href formula'].index:
    shark_df.loc[i,'href formula'] = shark_df.loc[i,'href formula'].replace(' ','',)

In [31]:
# revisamos otra vez para ver que más problemas puede haber en la columna
shark_df[shark_df['href formula'] != shark_df['href']].loc[:,['href','href formula']]

,href,href formula
27,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
134,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
180,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
263,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
264,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
271,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
272,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
293,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
323,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
361,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


In [32]:
# comprobamos si hay menos valores repetidos
(shark_df['href formula'] == shark_df['href']).value_counts()

True     5962
False      30
dtype: int64

In [33]:
shark_df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989


In [34]:
# seguimos revisando las diferencias entre las columnas..
# algunos de los registros no terminan en .pdf
# lo añadiremos si es necesario
shark_df[shark_df['href'].str.endswith('pdf') != True]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
157,2015.08.19,19-Aug-15,2015,Unprovoked,USA,Florida,"Jacksonville Beach, Duval County",Walking,Kaley Szarmack,F,...,N,15h30,3' shark,"ABC News, 8/21/2015",2015.08.19-Szsarmack,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.08.19,2015.08.19,5835
177,2015.06.30.b,30-Jun-15,2015,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Yamba",Surfing,Steve,M,...,N,NaN,NaN,"The Telegraph, 7/8/2015",2015.06.30.b-Steve.pf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.06.30.b,2015.06.30.b,5815
193,2015.06.13,13-Jun-15,2015,Unprovoked,USA,California,Off San Diego,NaN,Elke Specker,F,...,N,NaN,Mako shark,"Courthouse News Service, 11/18/2015",Court Case pending,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.06.13,2015.06.13,5799
226,2015.02.14,14-Feb-15,2015,Unprovoked,REUNION,dÉtang-Salé,Ravine Mula,Swimming,Talon Bishop,F,...,Y,18h30,"Tiger shark, 3.5 m","L'Yonne Républicaine, 2/14/2015",2015.02.14-Bishop.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.02.14,2015.02.14,5766
251,2014.11.20,20-Nov-14,2014,Provoked,MAURITIUS,Cargados Carajos Shoals (St. Brandon),NaN,Fishing,Rameshwar Ram Dhauro,M,...,Y,NaN,NaN,A. R. Ramjatan,2014.11.20-Mauritius.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.11.20,2014.11.20,5741
252,2014.11.19,19-Nov-14,2014,Boat,AUSTRALIA,Western Australia,Freo,Fishing,Boat: occupants: David Lock & his father,M,...,N,NaN,White shark,"The West Australian, 11/20/2014",2014.11.19-Freo.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.11.19,2014.11.19,5740
263,2014.10.18,18-Oct-14,2014,Unprovoked,USA,Hawaii,"Maalaea, South Maui",Surfing,Kaleo Roberson,M,...,N,11h15,12' to 14' shark,"Star Advertiser, 10/18/2014",2014.10.18-Roberson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.10.18,2014.10.18,5730
343,2014.04.12.R,Reported 12-Apr-2014,2014,Boat,SOUTH AFRICA,NaN,NaN,Shark watching,Inflatable boat,NaN,...,N,NaN,White shark,"You Tube, posted 4/12/2014",.2014.04.12.R-inflatable,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.04.12.R,2014.04.12.R,5650
352,2014.03.18.c,18-Mar-14,2014,Unprovoked,NEW CALEDONIA,Baie de Sainte-Marie,NaN,Kite Surfing,Laurent Borgna,M,...,N,17h58,NaN,"A. Brenneka, Shark Attack Surviors",2014.03.18.c-Borgna,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.03.18.c,2014.03.18.c,5641
359,2014.02.17,Reported 17-Feb-2014,2014,Boat,PAPUA NEW GUINEA,NaN,NaN,Sailing,OneDLL,M,...,N,Night,NaN,"Sail-World.com, 2/17/2014",2014.02.17.R-OneDLL,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.02.17,2014.02.17,5634


In [35]:
shark_df[shark_df['href formula'].str.endswith('pdf') != True]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
157,2015.08.19,19-Aug-15,2015,Unprovoked,USA,Florida,"Jacksonville Beach, Duval County",Walking,Kaley Szarmack,F,...,N,15h30,3' shark,"ABC News, 8/21/2015",2015.08.19-Szsarmack,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.08.19,2015.08.19,5835
177,2015.06.30.b,30-Jun-15,2015,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Yamba",Surfing,Steve,M,...,N,NaN,NaN,"The Telegraph, 7/8/2015",2015.06.30.b-Steve.pf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.06.30.b,2015.06.30.b,5815
193,2015.06.13,13-Jun-15,2015,Unprovoked,USA,California,Off San Diego,NaN,Elke Specker,F,...,N,NaN,Mako shark,"Courthouse News Service, 11/18/2015",Court Case pending,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.06.13,2015.06.13,5799
226,2015.02.14,14-Feb-15,2015,Unprovoked,REUNION,dÉtang-Salé,Ravine Mula,Swimming,Talon Bishop,F,...,Y,18h30,"Tiger shark, 3.5 m","L'Yonne Républicaine, 2/14/2015",2015.02.14-Bishop.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.02.14,2015.02.14,5766
251,2014.11.20,20-Nov-14,2014,Provoked,MAURITIUS,Cargados Carajos Shoals (St. Brandon),NaN,Fishing,Rameshwar Ram Dhauro,M,...,Y,NaN,NaN,A. R. Ramjatan,2014.11.20-Mauritius.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.11.20,2014.11.20,5741
252,2014.11.19,19-Nov-14,2014,Boat,AUSTRALIA,Western Australia,Freo,Fishing,Boat: occupants: David Lock & his father,M,...,N,NaN,White shark,"The West Australian, 11/20/2014",2014.11.19-Freo.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.11.19,2014.11.19,5740
343,2014.04.12.R,Reported 12-Apr-2014,2014,Boat,SOUTH AFRICA,NaN,NaN,Shark watching,Inflatable boat,NaN,...,N,NaN,White shark,"You Tube, posted 4/12/2014",.2014.04.12.R-inflatable,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.04.12.R,2014.04.12.R,5650
352,2014.03.18.c,18-Mar-14,2014,Unprovoked,NEW CALEDONIA,Baie de Sainte-Marie,NaN,Kite Surfing,Laurent Borgna,M,...,N,17h58,NaN,"A. Brenneka, Shark Attack Surviors",2014.03.18.c-Borgna,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.03.18.c,2014.03.18.c,5641
359,2014.02.17,Reported 17-Feb-2014,2014,Boat,PAPUA NEW GUINEA,NaN,NaN,Sailing,OneDLL,M,...,N,Night,NaN,"Sail-World.com, 2/17/2014",2014.02.17.R-OneDLL,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.02.17,2014.02.17,5634
363,2014.01.26,26-Jan-14,2014,Provoked,AUSTRALIA,New South Wales,Umina Beach,Fishing,Richard O'Connor,M,...,N,NaN,1m shark,"Daily Telegraph, 1/29/2014",2014.01.26-O'Connor.pf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.01.26,2014.01.26,5630


In [36]:
# usamos apply para añadir el sufijo pdf donde sea necesario
shark_df['href'] = shark_df['href'].apply(lambda x: x if x.endswith('.pdf') else x + '.pdf')

In [37]:
shark_df['href formula'] = shark_df['href formula'].apply(lambda x: x if x.endswith('.pdf') else x + '.pdf')

In [38]:
# comprobamos que todo está OK
shark_df[shark_df['href'].str.endswith('pdf') != True]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order


In [39]:
# seguimos revisando diferencias
shark_df[shark_df['href formula'] != shark_df['href']].loc[:,['href','href formula']]

,href,href formula
27,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
134,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
180,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
264,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
271,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
272,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
293,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
323,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
361,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
362,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


In [40]:
# quedan 20 valores a revisar
(shark_df['href formula'] == shark_df['href']).value_counts()

True     5972
False      20
dtype: int64

In [41]:
shark_df.loc[27,'href']

'http://sharkattackfile.net/spreadsheets/pdf_directory/2016.07.23-Cutbirth.pdf'

In [42]:
shark_df.loc[27,'href formula']

'http://sharkattackfile.net/spreadsheets/pdf_directory/2016.07.23.a-Cutbirth.pdf'

In [43]:
# vemos que algunos archivos tiene un caracter adicional
# nos quedaremos con el más largo en cada caso ya que es el correcto
for i in shark_df[shark_df['href formula'] != shark_df['href']].loc[:,['href','href formula']].index:
    if len(shark_df.loc[i,'href']) == len(shark_df.loc[i,'href formula']) - 2:
        shark_df.loc[i,'href'] = shark_df.loc[i,'href formula']
    else:
        shark_df.loc[i,'href formula'] = shark_df.loc[i,'href']
        


In [44]:
shark_df.loc[27,'href']

'http://sharkattackfile.net/spreadsheets/pdf_directory/2016.07.23.a-Cutbirth.pdf'

In [45]:
shark_df.loc[27,'href formula']

'http://sharkattackfile.net/spreadsheets/pdf_directory/2016.07.23.a-Cutbirth.pdf'

In [46]:
# comprobamos que queda por revisar
(shark_df['href formula'] == shark_df['href']).value_counts()

True    5992
dtype: int64

In [47]:
shark_df[shark_df['href formula'] != shark_df['href']].loc[:,['href','href formula']]

,href,href formula


In [48]:
# dado que las dos columnas son iguales
# dejaremos solo 1 de ellas
shark_df.drop(['href formula'],axis=1, inplace=True)

In [49]:
# revisamos otra vez estado general del df
shark_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5992 entries, 0 to 5991
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case Number             5992 non-null   object
 1   Date                    5992 non-null   object
 2   Year                    5992 non-null   int64 
 3   Type                    5992 non-null   object
 4   Country                 5949 non-null   object
 5   Area                    5590 non-null   object
 6   Location                5496 non-null   object
 7   Activity                5465 non-null   object
 8   Name                    5792 non-null   object
 9   Sex                     5425 non-null   object
 10  Age                     3311 non-null   object
 11  Injury                  5965 non-null   object
 12  Fatal (Y/N)             5992 non-null   object
 13  Time                    2779 non-null   object
 14  Species                 3058 non-null   object
 15  Inve

In [50]:
# revisamos las columnas Case Number 1 y 2 
# comparandolas con Case Number para intentar comprenderlas
shark_df[shark_df['Case Number'] == shark_df['Case Number.1']][['Case Number','Case Number.1','Case Number.2']]


,Case Number,Case Number.1,Case Number.2
0,2016.09.18.c,2016.09.18.c,2016.09.18.c
1,2016.09.18.b,2016.09.18.b,2016.09.18.b
2,2016.09.18.a,2016.09.18.a,2016.09.18.a
3,2016.09.17,2016.09.17,2016.09.17
5,2016.09.15.R,2016.09.15.R,2016.09.15.R
...,...,...,...
5987,ND.0005,ND.0005,ND.0005
5988,ND.0004,ND.0004,ND.0004
5989,ND.0003,ND.0003,ND.0003
5990,ND.0002,ND.0002,ND.0002


In [51]:
shark_df[shark_df['Case Number'] != shark_df['Case Number.1']][['Case Number','Case Number.1','Case Number.2']]

,Case Number,Case Number.1,Case Number.2
4,2016.09.15,2016.09.16,2016.09.15
33,2016.07.14.4,2016.07.14.R,2016.07.14.4
97,2016.01.24.b,2015.01.24.b,2016.01.24.b
116,2015.12.23,2015.11.07,2015.12.23
121,2015.10.28.a,2015.10.28,2015.10.28.a
169,2015.07-10,2015.07.10,2015.07.10
3296,1967.07.05,1967/07.05,1967.07.05
3569,"1962,08.30.b",1962.08.30.b,"1962,08.30.b"
3654,1961.09.02.R,"1961.09,06.R",1961.09.02.R
4177,1952.08.05,1952.08.04,1952.08.05


In [52]:
shark_df[shark_df['Case Number'] != shark_df['Case Number.2']][['Case Number','Case Number.1','Case Number.2']]


,Case Number,Case Number.1,Case Number.2
169,2015.07-10,2015.07.10,2015.07.10
4719,1934.01.08.R,1934.02.08.R,1934.02.08.R


In [53]:
# vemos que todos los valores excepto 15 son duplicados..
# la información que puede aportar es muy relativa
# en algunos casos se trata de pequeños cambios en el formato
shark_df.loc[3296][['Case Number','Case Number.1', 'Case Number.2']]

Case Number      1967.07.05
Case Number.1    1967/07.05
Case Number.2    1967.07.05
Name: 3296, dtype: object

In [54]:
# en otros se trata de pequeñas erratas
shark_df.loc[33][['Case Number','Case Number.1', 'Case Number.2']]

Case Number      2016.07.14.4
Case Number.1    2016.07.14.R
Case Number.2    2016.07.14.4
Name: 33, dtype: object

In [55]:
# al ser pocos casos los podemos tratar de forma personalizada
shark_df.loc[4,'Case Number'] = shark_df.loc[4,'Case Number.2']

In [56]:
shark_df.loc[33,'Case Number'],shark_df.loc[33,'Case Number.2'] = shark_df.loc[33,'Case Number.1'],shark_df.loc[33,'Case Number.1']

In [57]:
shark_df.loc[97,'Case Number.1'] = shark_df.loc[97,'Case Number']

In [58]:
shark_df.loc[116,'Case Number'],shark_df.loc[116,'Case Number.2'] = shark_df.loc[116,'Case Number.1'],shark_df.loc[116,'Case Number.1']

In [59]:
shark_df.loc[121,'Case Number.1'] = shark_df.loc[121,'Case Number.2']

In [60]:
shark_df.loc[169,'Case Number'] = shark_df.loc[169,'Case Number.1']

In [61]:
shark_df.loc[3296,'Case Number.1'] = shark_df.loc[3296,'Case Number']

In [62]:
shark_df.loc[3569,'Case Number'], shark_df.loc[3569,'Case Number.2'] = shark_df.loc[3569,'Case Number.1'],shark_df.loc[3569,'Case Number.1']


In [63]:
shark_df.loc[3654,'Case Number.1'] = shark_df.loc[3654,'Case Number']

In [64]:
shark_df.loc[4177,'Case Number.1'] = shark_df.loc[4177,'Case Number.2']

In [65]:
shark_df.loc[4719,'Case Number'] = shark_df.loc[4719,'Case Number.1']

In [66]:
shark_df.loc[5043,'Case Number'],shark_df.loc[5043,'Case Number.2'] = shark_df.loc[5043,'Case Number.1'], shark_df.loc[5043,'Case Number.1']


In [67]:
shark_df.loc[5150,'Case Number.1'] = shark_df.loc[5150,'Case Number']

In [68]:
# revisamos de nuevo
shark_df[shark_df['Case Number'] != shark_df['Case Number.2']][['Case Number','Case Number.1','Case Number.2']]


,Case Number,Case Number.1,Case Number.2


In [69]:
# dado que la información en las 3 columnas es la misma
# eliminamos las columnas Case Number.1 y Number.2
shark_df.drop(['Case Number.1','Case Number.2'],axis=1,inplace=True)

In [70]:
shark_df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5987,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,6
5988,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5
5989,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,4
5990,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,3


In [71]:
# revisamos otra vez estado general del df
shark_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5992 entries, 0 to 5991
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case Number             5992 non-null   object
 1   Date                    5992 non-null   object
 2   Year                    5992 non-null   int64 
 3   Type                    5992 non-null   object
 4   Country                 5949 non-null   object
 5   Area                    5590 non-null   object
 6   Location                5496 non-null   object
 7   Activity                5465 non-null   object
 8   Name                    5792 non-null   object
 9   Sex                     5425 non-null   object
 10  Age                     3311 non-null   object
 11  Injury                  5965 non-null   object
 12  Fatal (Y/N)             5992 non-null   object
 13  Time                    2779 non-null   object
 14  Species                 3058 non-null   object
 15  Inve

In [72]:
shark_df['original order']

0       5993
1       5992
2       5991
3       5990
4       5989
        ... 
5987       6
5988       5
5989       4
5990       3
5991       2
Name: original order, Length: 5992, dtype: int64

In [73]:
# usaremos la columna "original order" como nuevo indice
shark_df = shark_df.set_index(shark_df['original order']).sort_index()

In [74]:
shark_df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href,original order
original order,,,,,,,,,,,,,,,,,,,
2,ND.0001,1845-1853,0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2
3,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,3
4,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,4
5,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5
6,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,6


In [75]:
# dropeamos la columna original order ya que ya le hemos dado uso
shark_df.drop('original order',axis=1, inplace=True)

In [76]:
shark_df.head(), shark_df.tail()

(               Case Number         Date  Year        Type             Country  \
 original order                                                                  
 2                  ND.0001    1845-1853     0  Unprovoked  CEYLON (SRI LANKA)   
 3                  ND.0002    1883-1889     0  Unprovoked              PANAMA   
 4                  ND.0003    1900-1905     0  Unprovoked                 USA   
 5                  ND.0004  Before 1903     0  Unprovoked           AUSTRALIA   
 6                  ND.0005  Before 1903     0  Unprovoked           AUSTRALIA   
 
                              Area                             Location  \
 original order                                                           
 2                Eastern Province  Below the English fort, Trincomalee   
 3                             NaN                 Panama Bay 8ºN, 79ºW   
 4                  North Carolina                       Ocracoke Inlet   
 5               Western Australia               

In [77]:
# por alguna razón el indice empieza por 2, lo cual puede llevar a dudas
# ahora que ya tenemos bien el orden, haremos que el indice empiece en 0 como es habitual
shark_df.reset_index(inplace=True)
shark_df.drop('original order', axis=1, inplace=True)

In [78]:
# vemos el nuevo orden
shark_df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href
0,ND.0001,1845-1853,0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
1,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
2,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
3,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
4,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


In [79]:
# revisamos otra vez estado general del df
shark_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case Number             5992 non-null   object
 1   Date                    5992 non-null   object
 2   Year                    5992 non-null   int64 
 3   Type                    5992 non-null   object
 4   Country                 5949 non-null   object
 5   Area                    5590 non-null   object
 6   Location                5496 non-null   object
 7   Activity                5465 non-null   object
 8   Name                    5792 non-null   object
 9   Sex                     5425 non-null   object
 10  Age                     3311 non-null   object
 11  Injury                  5965 non-null   object
 12  Fatal (Y/N)             5992 non-null   object
 13  Time                    2779 non-null   object
 14  Species                 3058 non-null   object
 15  Inve

In [85]:
# revisamos las diferentes especies reportadas
species_list = []
for i in shark_df['Species'].unique():
    species_list.append(i)

In [104]:
len(species_list)

1539

In [107]:
# intentaremos  hacer una limpieza con regex
shark_species = re.compile('(^.*shark)')

In [109]:
for i in shark_df['Species'].index:
    # los valores nan los llamaremos Unknown species
    if isinstance(shark_df.loc[i,'Species'],float):
        shark_df.loc[i,'Species'] = 'Unknown species'
    # reduciremos las posibles nombres que terminen en shark
    # revisar si es necesario¿?
    elif shark_species.findall(shark_df.loc[i,'Species']):
        shark_df.loc[i,'Species'] = shark_species.findall(shark_df.loc[i,'Species'])[0]
    else:
        pass
        
      

In [111]:
# comprobamos si se ha reducido la lista
species_list_v2 = []
for i in shark_df['Species'].unique():
    species_list_v2.append(i)

In [112]:
len(species_list_v2)

869

In [121]:
species_list_v2

['Unknown species',
 'Said to involve a grey nurse shark',
 'Blue pointers',
 'Blue pointer',
 "12' tiger shark",
 'White shark',
 "6' shark",
 '"a black-tipped shark',
 'Bull shark',
 "1.2 m [4'] shark",
 'Nurse shark',
 'Tiger shark',
 '"A pack of shark',
 "1.8 m [6'] shark",
 "0.9 m [3'] shark",
 'Shovelnose guitarfish, adult male ',
 'A large hammerhead shark',
 'Silvertip shark',
 'Whale shark',
 'said to involve 2.5 m hammerhead shark',
 'Blacktip shark',
 'Basking shark',
 ' Galapagos shark',
 'Possibly white shark',
 '"The fish was harpooned, dried, and presented to the sailor, who went round Europe exhibiting it  It was said to be 20 feet long.',
 'Description of shark',
 'Thought to involve a porbeagle or mako shark',
 'Reported to involve a hammerhead shark',
 "Said to be a 7.6 m [25'] shark",
 '"a large shark',
 "18' shark",
 "Said to involved a 6 m to 7.3 m [20' to 24'] shark",
 '234-lb shark',
 'Remains recovered from shark',
 '4\' "ground shark',
 "16' shark",
 "3.2 m [1

In [140]:
# intentaremos agrupar los tipos más comunes
shark_white = re.compile('White|white')
shark_nurse = re.compile('Nurse|nurse')
shark_blue = re.compile('blue|Blue|Mako|mako')
shark_bull = re.compile('Bull|bull|leucas')
shark_hammer = re.compile('hammerhead|Hammerhead')
shark_whale = re.compile('Whale|whale|whaler')
shark_tiger = re.compile('Tiger|tiger')
shark_grey = re.compile('Grey|grey')
shark_black = re.compile('Blacktip|blacktip|black-tipped')
shark_wobbegong = re.compile('.*obbego.*')
shark_galapagos = re.compile('Galapagos|galapagos|galapago|Galapago')
shark_dusk = re.compile('macrurus|dusk|Dusk|Dusky|dusky')
shark_zambesi = re.compile('zambesi|Zambesi')
shark_sevengill = re.compile('Sevengill|sevengill')
shark_caribbean = re.compile('Caribbean|caribbean')
shark_sand = re.compile('Sand|sand')




In [141]:
shark_df_copy = shark_df.copy()

for i in shark_df.index:
    if len(shark_white.findall(shark_df_copy.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'White shark'
    elif len(shark_nurse.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Nurse shark'
    elif len(shark_blue.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Mako shark'
    elif len(shark_bull.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Bull shark'
    elif len(shark_hammer.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Hammerhead shark'
    elif len(shark_whale.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Whale shark'
    elif len(shark_tiger.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Tiger shark'
    elif len(shark_grey.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Hammerhead shark'
    elif len(shark_black.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Blacktip shark'
    elif len(shark_wobbegong.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Wobbegong shark'
    elif len(shark_galapagos.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Galapagos shark'
    elif len(shark_dusk.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Dusky shark'
    elif len(shark_zambesi.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Zambesi shark'
    elif len(shark_caribbean.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Caribbean shark'
    elif len(shark_sand.findall(shark_df.loc[i,'Species'])) > 0:
        shark_df_copy.loc[i,'Species'] = 'Sand shark'
    
    

In [142]:
# comprobamos si se ha reducido la lista
species_list_v3 = []
for i in shark_df_copy['Species'].unique():
    species_list_v3.append(i)

In [143]:
species_list_v3

['Unknown species',
 'Nurse shark',
 'Mako shark',
 'Tiger shark',
 'White shark',
 "6' shark",
 'Blacktip shark',
 'Bull shark',
 "1.2 m [4'] shark",
 '"A pack of shark',
 "1.8 m [6'] shark",
 "0.9 m [3'] shark",
 'Shovelnose guitarfish, adult male ',
 'Hammerhead shark',
 'Silvertip shark',
 'Whale shark',
 'Basking shark',
 'Galapagos shark',
 '"The fish was harpooned, dried, and presented to the sailor, who went round Europe exhibiting it  It was said to be 20 feet long.',
 'Description of shark',
 "Said to be a 7.6 m [25'] shark",
 '"a large shark',
 "18' shark",
 "Said to involved a 6 m to 7.3 m [20' to 24'] shark",
 '234-lb shark',
 'Remains recovered from shark',
 '4\' "ground shark',
 "16' shark",
 "3.2 m [10.5'] shark",
 '2 shark',
 "3' shark",
 '1.5 m shark',
 '"Shark caught later"',
 'Identified as C. gangeticus by Dr. J. Fayrer',
 'Wobbegong shark',
 '68" shark',
 "10' shark",
 '7-gill shark',
 "13' shark",
 "8' shark",
 "15' shark",
 ' a small shark',
 '2.5 m shark',
 '3 